# Purpose:

2015-02-16 (Monday)

Generate code to import and summarize collection field records in pandas while the real DB is being finished.

# Implementation:

## Imports:

In [1]:
# imports
import csv
import itertools

import pandas as pd
import numpy as np

import munch

from tribool import Tribool

In [2]:
# for plotting
import ggplot as g


## File paths:

In [3]:
# define paths to files
spring_summer = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/2014_spring_summer_from_rob.xlsx"
fall =          "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/2014_fall_for_pandas.xlsx"
december = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/DEC_2014_survey_for_pandas.xlsx"

h5_out_path = "/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/hdf5/2014_collection_records.h5"

In [4]:
village_id_map_path = "/home/gus/Dropbox/uganda_data/data_repos/field_data/locations/names/uganda_village_id_map.csv"

----
# Helper functions

In [5]:
def get_village_id_map(village_id_map_path):
    """
    Generates and returns a `dict` mapping the long-form village names to the letter codes.
    Letter codes map back to themselves to ensure a one way mapping.  
    Enforces both be all UPPERcase to allow case insensitivity as long as
    the map is used like: `map[text.upper()]`.
    :return: `dict`
    """

    village_id_map = {}

    with open(village_id_map_path, 'rb') as csv_file:
        village_ids = csv.reader(csv_file, delimiter=',')
        for pair in village_ids:
            village_id_map[unicode(pair[0].upper())] = unicode(pair[0].upper())
            village_id_map[unicode(pair[1].upper())] = unicode(pair[0].upper())

    return village_id_map

In [6]:
def load_xl_sheets(xl_path):
    dfs = munch.Munch()
    
    xls = pd.ExcelFile(xl_path)
    
    for sheet in xls.sheet_names:
        if sheet.upper().startswith("DISSECT"):
            dfs[sheet] = xls.parse(sheetname=sheet, 
                                header=0, 
                                skiprows=None, skip_footer=0, 
                                index_col=None, parse_cols=None, 
                                parse_dates=False, date_parser=None, 
                                na_values=['NA'], 
                                thousands=None, chunksize=None, 
                                convert_float=False, 
                                has_index_names=False, converters=None)
    return dfs

In [7]:
def recode_villages(df):
    map_func = lambda x: village_id_map[x.upper()]
        
    new_codes = df.Village.apply(map_func)
    df.Village = new_codes    

In [8]:
def recode_dead(df):
    def recode_func(x):
        # this is treated as an unknown case
        if pd.isnull(x):
            return x

        x = unicode(x)

        # True means DEAD
        # False means LIVE or NOT-DEAD
        # None means unknown

        try:
            # deal with Live type cases
            if x.upper().startswith('L'):
                return False


            if x.startswith('0'):
                return False


            # deal with Dead type cases
            if x.upper().startswith('D'):
                return True


            if x.startswith('1'):
                return True


            # deal with unknown type cases
            if x.upper().startswith('UN'):
                return None
        except AttributeError:
            return x

        msg = "The value {x} was not expected and this function must be corrected to continue.".format(x=x)
        raise ValueError(msg)

    new_dead = df.Dead.apply(recode_func)
    df.Dead = new_dead

##########################################

def recode_teneral(df):
    def recode_func(x):

        # this is treated as an unknown case
        if pd.isnull(x):
            return x

        x = unicode(x)

        # True means teneral
        # False means NOT-teneral
        # None means unknown

        try:
            # deal with NOT-teneral type cases
            if x.upper().startswith('N'):
                return False

            if x.startswith('0'):
                return False

            # deal with Teneral type cases
            if x.upper().startswith('T'):
                return True
            
            if x.startswith('1'):
                return True


            # Deal with unknown type cases
            if x.upper().startswith('UN'):
                return x
        except AttributeError:
            return x

        msg = "The value {x} was not expected and this function must be corrected to continue.".format(x=x)
        raise ValueError(msg)
    
    
    new_teneral = df.Teneral.apply(recode_func)
    df.Teneral = new_teneral

##########################################

def recode_positives(df):
    def recode_func(x):
        # this is treated as an unknown case
        if pd.isnull(x):
            return x

        y = unicode(x)

        # deal with Unknown type cases
        if y.upper().startswith('UN'):
            return None

        if y.upper().startswith('DEAD'):
            return None


        # deal with Positive type cases
        if y.startswith('1'):
            return True


        if y.upper().startswith('TRUE'):
            return True

        if y.upper().startswith('P'):
            return True

        if y.upper().startswith('Y'):
            return True


        # deal with Negative type cases
        if y.upper().startswith('NO'):
            return False

        if y.upper().startswith('FALSE'):
            return False


        if y.startswith('0'):
            return False


        msg = "The value {x} was not expected and this function must be corrected to continue.".format(x=x)
        raise ValueError(msg)


    new_prob = df.prob.apply(recode_func)
    df.prob = new_prob
    
    new_midgut = df.midgut.apply(recode_func)
    df.midgut = new_midgut
    
    new_sal_gland = df.sal_gland.apply(recode_func)
    df.sal_gland = new_sal_gland

##########################################

def recode_species(df):

    recode_func = lambda x: ''.join(x.split('.')).capitalize()

    new_Species = df.Species.apply(recode_func)
    df.Species = new_Species

##########################################

def recode_sex(df):

    recode_func = lambda x: x.upper()

    new_Sex = df.Sex.apply(recode_func)
    df.Sex = new_Sex

In [9]:
def aggregate_column_from_df_list(df_list, col_name):
    agg_data = []
    for df in df_list:
        agg_data.extend(list(df[col_name]))
        
    
    return agg_data

### Functions that add new columns

In [10]:
def add_infection_state_col(df):
    
    df['infection_state'] = df[['prob','midgut','sal_gland']].any(skipna=True,axis=1)
    
        

----
# Helpful constants

In [11]:
village_id_map = get_village_id_map(village_id_map_path)

In [12]:
# pdb

In [13]:
spring_summer_dfs = load_xl_sheets(spring_summer)
fall_dfs = load_xl_sheets(fall)
december_dfs = load_xl_sheets(december)

In [14]:
fall_dfs.values()[0].head(2)

,Village,Trap_No,Date,Species,Sex,Teneral,Dead,Fly_Number,Hunger_stage,prob,midgut,sal_gland,Wing_fray,Kept_in,Comment
0,Kitgum town council,1,2014-10-08,Gff,M,T,NaN,1,NaN,0,0,0,NaN,Ethanol,NaN
1,Kitgum town council,1,2014-10-08,Gff,M,NT,NaN,2,3,0,0,0,2,Ethanol,NaN


In [15]:
december_dfs.values()[0].head(3)

,Village,Trap_No,Date,Species,Sex,Teneral,Dead,Fly_Number,Hunger_stage,prob,midgut,sal_gland,Wing_fray,Kept_in,Comment
0,Ocala,11,2014-12-15,Gff,F,NT,NaN,1,2,0,0,0,2,Ethanol,NaN
1,Ocala,11,2014-12-15,Gff,M,NT,NaN,2,2,0,0,0,2,Ethanol,NaN
2,Ocala,10,2014-12-15,Gff,M,NT,NaN,3,2,0,0,0,2,Ethanol,No MG


In [16]:
# pdb

In [17]:
# list all dfs and run our recoder's so far
df_all = spring_summer_dfs.values() + fall_dfs.values() + december_dfs.values()

for df in df_all:
    recode_sex(df)
    recode_species(df)
    recode_villages(df)
    recode_positives(df)
    recode_dead(df)
    recode_teneral(df)
    add_infection_state_col(df)

In [18]:
df.head()

,Village,Trap_No,Date,Species,Sex,Teneral,Dead,Fly_Number,Hunger_stage,prob,midgut,sal_gland,Wing_fray,Kept_in,Comment,infection_state
0,OCU,6,2014-12-20,Gff,M,False,False,1,2,False,False,False,3,Ethanol,0,False
1,OCU,6,2014-12-20,Gff,M,False,False,2,1,False,False,False,2,Ethanol,0,False
2,OCU,6,2014-12-20,Gff,M,False,False,3,1,False,False,False,2,Ethanol,0,False
3,OCU,6,2014-12-20,Gff,M,False,False,4,2,False,False,False,2,Ethanol,0,False
4,OCU,6,2014-12-20,Gff,M,False,False,5,1,False,False,False,2,Ethanol,0,False


In [19]:
df_all[2].head()

,Village,Trap_No,Date,Species,Sex,Teneral,Dead,Fly_Number,Hunger_stage,Wing_fray,prob,midgut,sal_gland,Kept_in,Comment,infection_state
0,GOR,2,2014/07/27,Gff,F,False,False,GOR-14 001,2,2,False,False,False,EtOH,NaN,False
1,GOR,2,2014/07/27,Gff,F,False,False,GOR-14 002,2,2,False,False,False,EtOH,NaN,False
2,GOR,2,2014/07/27,Gff,M,False,False,GOR-14 003,3,2,False,False,False,EtOH,NaN,False
3,GOR,2,2014/07/27,Gff,F,False,False,GOR-14 004,3,2,False,False,False,EtOH,NaN,False
4,GOR,2,2014/07/27,Gff,F,False,False,GOR-14 005,2,2,False,False,False,EtOH,NaN,False


In [20]:
(df_all[2].prob | df_all[2].midgut | df_all[2].sal_gland).sum()

0

In [21]:
prob_all = aggregate_column_from_df_list(df_all, 'prob')

prob_all.count(True)

0

In [22]:
midgut_all = aggregate_column_from_df_list(df_all, 'midgut')

midgut_all.count(True)

83

In [23]:
sal_gland_all = aggregate_column_from_df_list(df_all, 'sal_gland')

sal_gland_all.count(True)

3

In [24]:
infection_state_all = aggregate_column_from_df_list(df_all, 'infection_state')

infection_state_all.count(True)

85

---
# Combine dataframes to one for ruling

In [25]:
df_big = pd.concat(df_all)

In [26]:
len(df_big)

6956

In [27]:
df_big.head()

,Comment,Date,Dead,Fly_Number,Hunger_stage,Kept_in,Sex,Species,Teneral,Trap_No,Village,Wing_fray,infection_state,midgut,prob,sal_gland
0,NaN,2014/07/22,False,UWA-14 001,2,EtOH,F,Gff,False,1,UWA,2,False,False,False,False
1,NaN,2014/07/22,False,UWA-14 002,3,EtOH,F,Gff,False,1,UWA,2,False,False,False,False
2,NaN,2014/07/22,False,UWA-14 003,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False
3,NaN,2014/07/22,False,UWA-14 004,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False
4,NaN,2014/07/22,False,UWA-14 005,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False


---
# Save this table

In [28]:
# df_big.to_hdf(path_or_buf=h5_out_path, key='main', mode='w')

---
# Pivot this table

In [29]:
# df1 = df_big.query('(Species == ["Gff"])').query('Village == ["DUK", "ORB", "ACA", "LIB"]')
df1 = df_big.query('(Species == ["Gff"])').query('Village == ["UWA"]')

In [30]:
df1

,Comment,Date,Dead,Fly_Number,Hunger_stage,Kept_in,Sex,Species,Teneral,Trap_No,Village,Wing_fray,infection_state,midgut,prob,sal_gland
0,NaN,2014/07/22,False,UWA-14 001,2,EtOH,F,Gff,False,1,UWA,2,False,False,False,False
1,NaN,2014/07/22,False,UWA-14 002,3,EtOH,F,Gff,False,1,UWA,2,False,False,False,False
2,NaN,2014/07/22,False,UWA-14 003,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False
3,NaN,2014/07/22,False,UWA-14 004,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False
4,NaN,2014/07/22,False,UWA-14 005,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False
5,NaN,2014/07/22,False,UWA-14 006,2,EtOH,M,Gff,False,1,UWA,2,False,False,False,False
6,NaN,2014/07/22,False,UWA-14 007,3,EtOH,F,Gff,False,1,UWA,2,False,False,False,False
7,NaN,2014/07/22,False,UWA-14 008,3,EtOH,F,Gff,False,1,UWA,2,False,False,False,False
8,NaN,2014/07/22,False,UWA-14 009,3,EtOH,F,Gff,False,1,UWA,3,False,False,False,False
9,NaN,2014/07/22,False,UWA-14 010,2,EtOH,F,Gff,False,1,UWA,2,False,False,False,False


In [31]:

table1 = pd.pivot_table(df1,index=['Village', 'Fly_Number', 'Date','Dead'],
                      values=['Species'],
                      fill_value=0,
                      aggfunc=[len])
table1

len
                                    Species
Village Fly_Number Date       Dead         
UWA     UWA-14 001 2014/07/22 False       1
        UWA-14 002 2014/07/22 False       1
        UWA-14 003 2014/07/22 False       1
        UWA-14 004 2014/07/22 False       1
        UWA-14 005 2014/07/22 False       1
        UWA-14 006 2014/07/22 False       1
        UWA-14 007 2014/07/22 False       1
        UWA-14 008 2014/07/22 False       1
        UWA-14 009 2014/07/22 False       1
        UWA-14 010 2014/07/22 False       1
        UWA-14 011 2014/07/22 False       1
        UWA-14 012 2014/07/22 False       1
        UWA-14 013 2014/07/22 False       1
        UWA-14 014 2014/07/22 False       1
        UWA-14 015 2014/07/22 False       1
        UWA-14 016 2014/07/22 False       1
        UWA-14 017 2014/07/22 False       1
        UWA-14 018 2014/07/22 False       1
        UWA-14 019 2014/07/22 True        1
        UWA-14 020 2014/07/22 True        1
        UWA-14 021 2014/07/22 True        1
        UWA-14 022 2014/07/22 True        1
        UWA-14 023 2014/07/22 True        1
        UWA-14 024 2014/07/22 False       1
        UWA-14 025 2014/07/22 False       1
        UWA-14 026 2014/07/22 False       1
        UWA-14 027 2014/07/22 False       1
        UWA-14 028 2014/07/22 False       1
        UWA-14 029 2014/07/22 False       1
        UWA-14 030 2014/07/22 False       1
...                                     ...
        UWA-14 749 2014/07/24 True        1
        UWA-14 750 2014/07/24 True        1
        UWA-14 751 2014/07/24 True        1
        UWA-14 752 2014/07/24 True        1
        UWA-14 753 2014/07/24 True        1
        UWA-14 754 2014/07/24 True        1
        UWA-14 755 2014/07/24 True        1
        UWA-14 756 2014/07/24 True        1
        UWA-14 757 2014/07/24 True        1
        UWA-14 758 2014/07/24 True        1
        UWA-14 759 2014/07/24 False       1
        UWA-14 760 2014/07/24 False       1
        UWA-14 761 2014/07/24 False       1
        UWA-14 762 2014/07/24 True        1
        UWA-14 763 2014/07/24 True        1
        UWA-14 764 2014/07/24 True        1
        UWA-14 765 2014/07/24 True        1
        UWA-14 766 2014/07/24 True        1
        UWA-14 767 2014/07/24 True        1
        UWA-14 768 2014/07/24 True        1
        UWA-14 769 2014/07/24 True        1
        UWA-14 770 2014/07/24 True        1
        UWA-14 771 2014/07/24 True        1
        UWA-14 772 2014/07/24 True        1
        UWA-14 773 2014/07/24 True        1
        UWA-14 774 2014/07/24 True        1
        UWA-14 775 2014/07/24 True        1
        UWA-14 776 2014/07/24 True        1
        UWA-14 777 2014/07/24 True        1
        UWA-14 778 2014/07/24 True        1

[729 rows x 1 columns]

In [32]:
table1.to_csv('/home/gus/Documents/YalePostDoc/project_stuff/g_f_fucipes_uganda/collection_data/village_data/UWA_2014.csv')

In [54]:
table2 = pd.pivot_table(df_big.query('Species == ["Gff"]'),index=['Village'],
                      values=['Count'],
                      columns=['infection_state'],
                      fill_value=0,
                      aggfunc=[len])
table2

len      
infection_state False True 
Village                    
ABO                 9     0
ACA              1333    21
AG                 41     0
AIN               173     0
AKA               679     6
AKY                31     0
ALE                44     0
ALI                23     0
AMI                67     1
APU                96     1
ASW                 6     0
ATM                 7     0
BOL               100     1
CE                 11     0
CHU                91     0
DUK               192     2
GAN               120     1
GOR               169     0
JIA                45     0
KIL                26     0
KTC                56     1
LAG                17     0
LEA                24     0
LIB                80     1
MOP                 1     0
MWA               309     0
NGO                72     0
OCA               565    12
OCL                 2     0
OCU               577    22
OD                368    12
OGU                34     0
OKS                74     0
OLE                80     0
OLO                82     0
OLW                 2     0
OMI                15     0
OPU                23     0
ORB               124     0
ORI                 4     0
ORV                31     0
OSG                98     0
PAW                12     0
TEO                 1     0
TUM                41     1
UWA               726     3
WEN               101     0

In [41]:
gff_2014 = df_big.query('Species == ["Gff"]')

In [43]:
#how many villages have we sampled in 2014?
len(gff_2014.Village.unique())

47

In [44]:
# How many total files collected?
len(gff_2014.Fly_Number)

6867

In [47]:
# flies dissected?
sum(gff_2014.Dead == False)

3699

In [99]:
# Positives?
set(gff_2014.infection_state == True)

{False, True}

In [72]:
# positives per dissected fly
85./3699*100

2.297918356312517

In [73]:
# best villages regarding possitives per dissected
table3 = pd.pivot_table(gff_2014.query("Dead == False"),index=['Village'],
                      values=['Count'],
                      columns=['infection_state'],
                      fill_value=0,
                      aggfunc=[len])
table3.head()

len      
infection_state False True 
Village                    
ABO                 5     0
ACA               524    12
AG                 17     0
AIN               141     0
AKA               377     5

In [65]:
pos_per_live = table3['len',True]/table3['len',False]

In [69]:
pos_per_live.sort(ascending=False)

In [71]:
pos_per_live * 100

Village
OCU        6.321839
OD         6.217617
OCA        3.832753
ACA        2.290076
AMI        2.222222
DUK        1.626016
AKA        1.326260
GAN        1.265823
APU        1.041667
UWA        0.632911
OLW        0.000000
OKS        0.000000
OLE        0.000000
OLO        0.000000
OSG        0.000000
OPU        0.000000
ORB        0.000000
ORI        0.000000
ORV        0.000000
MWA        0.000000
OGU        0.000000
AG         0.000000
LEA        0.000000
JIA        0.000000
MOP        0.000000
WEN        0.000000
AIN        0.000000
AKY        0.000000
ALE        0.000000
CE         0.000000
GOR        0.000000
ALI        0.000000
ABO        0.000000
dtype: float64

In [105]:
# infections by sex
table4 = pd.pivot_table(gff_2014.query("infection_state == True"),index=['Dead'],
                      values=[],
                      columns=['infection_state','Sex'],
                      margins=True,
                      fill_value=0,
                      aggfunc=[len])
table4

len        
infection_state True     All
Sex                F   M    
Dead                        
False             44  26  70
All               57  28  85

In [95]:
table4

len           
infection_state False True   All
Village Sex                     
ABO     M           5    0     5
ACA     F         351   10   361
        M         173    2   175
AG      F          17    0    17
AIN     F          86    0    86
        M          55    0    55
AKA     F         255    3   258
        M         122    2   124
AKY     F          17    0    17
        M           2    0     2
ALE     F          12    0    12
        M          19    0    19
ALI     F          14    0    14
        M           2    0     2
AMI     F          24    1    25
        M          21    0    21
APU     F          75    1    76
        M          21    0    21
CE      F           3    0     3
        M           2    0     2
DUK     F          59    2    61
        M          64    0    64
GAN     F          60    1    61
        M          19    0    19
GOR     F          87    0    87
        M          20    0    20
JIA     F          19    0    19
        M          11    0    11
LEA     F           9    0     9
        M           7    0     7
...               ...  ...   ...
MWA     M         109    0   109
OCA     F         203    4   207
        M          84    7    91
OCU     F         216   11   227
        M         132   11   143
OD      F         141    9   150
        M          52    3    55
OGU     F          21    0    21
        M           7    0     7
OKS     F          33    0    33
        M          13    0    13
OLE     F          36    0    36
        M          18    0    18
OLO     F          42    0    42
        M          20    0    20
OLW     F           2    0     2
OPU     F          13    0    13
        M           3    0     3
ORB     F          75    0    75
        M          25    0    25
ORI     F           3    0     3
ORV     F           8    0     8
        M          13    0    13
OSG     F          55    0    55
        M          17    0    17
UWA     F         288    2   290
        M         186    1   187
WEN     F          45    0    45
        M          23    0    23
All              3629   70  3699

[62 rows x 3 columns]

In [75]:
gff_2014.

,Comment,Date,Dead,Fly_Number,Hunger_stage,Kept_in,Sex,Species,Teneral,Trap_No,Village,Wing_fray,infection_state,midgut,prob,sal_gland
0,NaN,2014/07/22,False,UWA-14 001,2,EtOH,F,Gff,False,1,UWA,2,False,False,False,False
1,NaN,2014/07/22,False,UWA-14 002,3,EtOH,F,Gff,False,1,UWA,2,False,False,False,False
2,NaN,2014/07/22,False,UWA-14 003,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False
3,NaN,2014/07/22,False,UWA-14 004,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False
4,NaN,2014/07/22,False,UWA-14 005,3,EtOH,M,Gff,False,1,UWA,2,False,False,False,False
